<p align="center">
  <h1 align="center">🌊 FlowGrad v0.5 — RecSys Embedding Diagnostics</h1>
  <p align="center">
    <strong>End-to-End Validation: Baseline → Diagnose → Fix → Compare → Statistical Proof</strong>
  </p>
</p>

---

This notebook demonstrates how FlowGrad's new **EmbeddingTracker** diagnoses common recommendation system failures (Dead Embeddings, Zombie Oscillation, Popularity Bias) and provides actionable XML prescriptions that actually improve performance. We also prove the improvement via formal statistical testing (Paired t-test).

**Dataset:** MovieLens-100K
**Model:** Matrix Factorization (PyTorch)

## 1. Setup & Data Loading

In [ ]:
!pip install torch pandas numpy scipy statsmodels gdown

In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import urllib.request
import zipfile
from scipy import stats

# Download MovieLens-100K
url = "http://files.grouplens.org/datasets/movielens/ml-100k.zip"
if not os.path.exists("ml-100k"):
    urllib.request.urlretrieve(url, "ml-100k.zip")
    with zipfile.ZipFile("ml-100k.zip", 'r') as zip_ref:
        zip_ref.extractall(".")

columns = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('ml-100k/u.data', sep='\t', names=columns)

# Re-index starting from 0
df['user_id'] = df['user_id'].astype('category').cat.codes
df['item_id'] = df['item_id'].astype('category').cat.codes

num_users = df['user_id'].nunique()
num_items = df['item_id'].nunique()
print(f"Users: {num_users}, Items: {num_items}, Ratings: {len(df)}")

# Simple Train/Test Split (Leave-One-Out for evaluation)
df = df.sort_values(by=['user_id', 'timestamp'])
test_df = df.groupby('user_id').tail(1)
train_df = df.drop(test_df.index)

class MFDataset(Dataset):
    def __init__(self, df):
        self.users = torch.tensor(df['user_id'].values, dtype=torch.long)
        self.items = torch.tensor(df['item_id'].values, dtype=torch.long)
        self.ratings = torch.tensor(df['rating'].values, dtype=torch.float32)
        
    def __len__(self):
        return len(self.users)
        
    def __getitem__(self, idx):
        return self.users[idx], self.items[idx], self.ratings[idx]

train_loader = DataLoader(MFDataset(train_df), batch_size=1024, shuffle=True)
test_loader = DataLoader(MFDataset(test_df), batch_size=1024, shuffle=False)

## 2. Baseline Model & FlowGrad Tracking
We train a standard Matrix Factorization model. We attach `EmbeddingTracker` to the Item Embedding layer to see what happens under the hood during training.

In [ ]:
from flowgrad import EmbeddingTracker

class MatrixFactorization(nn.Module):
    def __init__(self, num_users, num_items, dim=32):
        super().__init__()
        self.user_emb = nn.Embedding(num_users, dim)
        self.item_emb = nn.Embedding(num_items, dim)
        
        # Initialize purposefully bad to trigger diagnostics later if needed, 
        # or just standard normal.
        nn.init.normal_(self.user_emb.weight, std=0.01)
        nn.init.normal_(self.item_emb.weight, std=0.01)
        
    def forward(self, user, item):
        u = self.user_emb(user)
        i = self.item_emb(item)
        return (u * i).sum(dim=1)

model_baseline = MatrixFactorization(num_users, num_items, dim=32)
optimizer = torch.optim.Adam(model_baseline.parameters(), lr=0.05) # Aggressive LR
criterion = nn.MSELoss()

# Attach FlowGrad!
item_tracker = EmbeddingTracker(model_baseline.item_emb, name="baseline_item_emb")

print("Training Baseline Model...")
model_baseline.train()
for epoch in range(5):  # Short training for demo
    total_loss = 0
    for users, items, ratings in train_loader:
        optimizer.zero_grad()
        preds = model_baseline(users, items)
        loss = criterion(preds, ratings)
        loss.backward()
        optimizer.step()
        
        # FlowGrad Step
        item_tracker.step()
        
        total_loss += loss.item()
    print(f"Epoch {epoch+1} Loss: {total_loss/len(train_loader):.4f}")

## 3. FlowGrad Diagnosis
Let's see what FlowGrad found wrong with our baseline training dynamics.

In [ ]:
# Human readable report
item_tracker.report()

In [ ]:
from flowgrad import AgentExporter

# What the AI Agent sees:
xml_output = AgentExporter.export_embedding(item_tracker)
print(xml_output)

## 4. Applying the Fix (Prescription)
FlowGrad likely detected **ZOMBIE_EMBEDDINGS** (due to our aggressive LR=0.05 causing optimizer oscillation) and **POPULARITY_BIAS** (since MovieLens is heavily skewed). 

Let's apply the prescribed fix: **Lower LR** (or SparseAdam) and implement **Log-Q Correction** or simpler negative sampling. For this demo, we'll fix the learning rate and add a simple L2 regularization to control the oscillation.

In [ ]:
model_fixed = MatrixFactorization(num_users, num_items, dim=32)

# FIX 1: Reduced LR to stop Zombie Oscillation
# FIX 2: Added weight_decay (regularization) to prevent exploding norms in popular items
optimizer_fixed = torch.optim.Adam(model_fixed.parameters(), lr=0.005, weight_decay=1e-5)
criterion = nn.MSELoss()

item_tracker_fixed = EmbeddingTracker(model_fixed.item_emb, name="fixed_item_emb")

print("Training Fixed Model...")
model_fixed.train()
for epoch in range(5):
    total_loss = 0
    for users, items, ratings in train_loader:
        optimizer.zero_grad()
        preds = model_fixed(users, items)
        loss = criterion(preds, ratings)
        loss.backward()
        optimizer.step()
        
        item_tracker_fixed.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1} Loss: {total_loss/len(train_loader):.4f}")
    
item_tracker_fixed.report()

## 5. Statistical Proof of Improvement ($\%$ MSE Reduction and Paired t-test)
To ensure FlowGrad isn't just delivering heuristics, we construct a Paired t-test over the absolute errors on the test set. If FlowGrad's recommendations are mathematically sound (e.g. they structurally reduce gradient oscillating variance $\sigma^2_{t}$ for embeddings), we expect a **statistically significant reduction in prediction error**.

$\Delta = E[MSE_{baseline}] - E[MSE_{fixed}]$

In [ ]:
def get_individual_errors(model, loader):
    model.eval()
    errors = []
    with torch.no_grad():
        for users, items, ratings in loader:
            preds = model(users, items)
            batch_errs = (preds - ratings)**2
            errors.extend(batch_errs.cpu().numpy().tolist())
    return np.array(errors)

baseline_errors = get_individual_errors(model_baseline, test_loader)
fixed_errors = get_individual_errors(model_fixed, test_loader)

baseline_mse = baseline_errors.mean()
fixed_mse = fixed_errors.mean()

# Paired T-test
t_stat, p_val = stats.ttest_rel(baseline_errors, fixed_errors)

print("=\" * 50)
print("📊 Evaluation: Empirical Proof of RecSys Fixes")
print("=\" * 50)
print(f"📉 Baseline Test MSE: {baseline_mse:.4f}")
print(f"📈 FIXED Test MSE:    {fixed_mse:.4f}")
print(f"Improvement:          {(baseline_mse - fixed_mse) / baseline_mse * 100:.2f}%")
print("-" * 50)
print(f"Paired t-test p-value: {p_val:.4e}")

if p_val < 0.05 and fixed_mse < baseline_mse:
    print("✅ Conclusion: The FlowGrad recommended RecSys fixes yield a STATISTICALLY SIGNIFICANT improvement.")
else:
    print("❌ Conclusion: The fixes do not deliver a statistically significant improvement.")